In [3]:
import os
import glob
import random
import shutil
from tqdm import tqdm

# 훈련, 검증 세트 분할
def split_data(src_dir, train_dst_dir, val_dst_dir, train_ratio):
    # 지정된 디렉토리에서 이미지 파일과 JSON 파일 목록을 가져옵니다.
    img_files = glob.glob(src_dir + '/*.jpg')
    json_files = glob.glob(src_dir + '/*.json')

    # 파일 목록을 무작위로 섞습니다.
    random.shuffle(img_files)

    # 훈련 데이터의 개수를 계산합니다.
    num_train = int(len(img_files) * train_ratio)

    # 파일 목록을 훈련용과 검증용으로 분할합니다.
    train_files = img_files[:num_train]
    val_files = img_files[num_train:]

    # 훈련용 데이터를 복사합니다.
    for filepath in tqdm(train_files, desc='Copying files'):
        filename = os.path.basename(filepath)
        shutil.copy2(filepath, os.path.join(train_dst_dir, filename))  # 이미지 파일 복사
        shutil.copy2(filepath.replace('.jpg', '.json'), os.path.join(train_dst_dir, filename.replace('.jpg', '.json')))  # JSON 파일 복사

    # 검증용 데이터를 복사합니다.
    for filepath in tqdm(val_files, desc='Copying files'):
        filename = os.path.basename(filepath)
        shutil.copy2(filepath, os.path.join(val_dst_dir, filename))  # 이미지 파일 복사
        shutil.copy2(filepath.replace('.jpg', '.json'), os.path.join(val_dst_dir, filename.replace('.jpg', '.json')))  # JSON 파일 복사

# 실행 예시
src_dir = 'D:/Study/TeamProject_2/dataset_filtered'  # 원본 데이터셋 경로
train_dst_dir = 'D:/Study/TeamProject_2/dataset_filtered/train'  # 훈련 데이터셋 경로
val_dst_dir = 'D:/Study/TeamProject_2/dataset_filtered/val'  # 검증 데이터셋 경로
train_ratio = 0.8  # 훈련 데이터의 비율

# 훈련용 및 검증용 폴더가 없을 경우 생성합니다.
os.makedirs(train_dst_dir, exist_ok=True)
os.makedirs(val_dst_dir, exist_ok=True)

split_data(src_dir, train_dst_dir, val_dst_dir, train_ratio)

Copying files: 100%|██████████| 5335/5335 [04:50<00:00, 18.39it/s]


In [6]:
import tensorflow as tf
import json

def create_tf_example(json_path, img_path):
    with open(json_path, 'r') as json_file:
        data = json.load(json_file)

    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()

    width = data['description']['imageWidth']
    height = data['description']['imageHeight']

    filename = data['info']['video_id'].encode('utf8')
    image_format = b'jpg'

    xmins = [] 
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    # environment annotations
    for annotation in data['annotations']['environment']:
        points = annotation['points']
        x_values = [point[0] for point in points]
        y_values = [point[1] for point in points]
        xmins.append(min(x_values) / width)
        xmaxs.append(max(x_values) / width)
        ymins.append(min(y_values) / height)
        ymaxs.append(max(y_values) / height)
        classes_text.append(("environment_" + annotation['area_code']).encode('utf8'))
        # TODO: Add class ID conversion here
        # classes.append(class_to_id_dict[annotation['class']])

    # PM annotations
    for annotation in data['annotations']['PM']:
        points = annotation['points']
        xmins.append(points[0] / width)
        xmaxs.append((points[0] + points[2]) / width)
        ymins.append(points[1] / height)
        ymaxs.append((points[1] + points[3]) / height)
        classes_text.append(("PM_" + annotation['PM_code']).encode('utf8'))
        # TODO: Add class ID conversion here
        # classes.append(class_to_id_dict[annotation['class']])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))
    return tf_example

Processing data: 0it [00:00, ?it/s]


In [8]:
from glob import glob

# 훈련 데이터 TFRecord 파일 생성
# 이미지와 JSON 파일이 위치한 디렉토리
dir_path = 'D:/Study/TeamProject_2/dataset_filtered/train'

# JSON 파일 경로들을 가져옵니다
json_paths = glob(os.path.join(dir_path, '*.json'))

# TFRecord를 저장할 writer를 생성합니다
writer = tf.io.TFRecordWriter('train.tfrecord')

for json_path in tqdm(json_paths, desc="Processing data"):
    # JSON 파일과 같은 이름의 이미지 파일 경로를 가져옵니다
    img_path = os.path.splitext(json_path)[0] + '.jpg'
    
    # JSON과 이미지 파일을 이용하여 TFRecord를 생성하고, 이를 파일에 씁니다
    tf_example = create_tf_example(json_path, img_path)
    writer.write(tf_example.SerializeToString())

writer.close()

Processing data: 100%|██████████| 21336/21336 [17:36<00:00, 20.20it/s]


In [10]:
# 검증 데이터 TFRecord 파일 생성
# 이미지와 JSON 파일이 위치한 디렉토리
dir_path = 'D:/Study/TeamProject_2/dataset_filtered/val'

# JSON 파일 경로들을 가져옵니다
json_paths = glob(os.path.join(dir_path, '*.json'))

# TFRecord를 저장할 writer를 생성합니다
writer = tf.io.TFRecordWriter('val.tfrecord')

for json_path in tqdm(json_paths, desc="Processing data"):
    # JSON 파일과 같은 이름의 이미지 파일 경로를 가져옵니다
    img_path = os.path.splitext(json_path)[0] + '.jpg'
    
    # JSON과 이미지 파일을 이용하여 TFRecord를 생성하고, 이를 파일에 씁니다
    tf_example = create_tf_example(json_path, img_path)
    writer.write(tf_example.SerializeToString())

writer.close()

Processing data: 100%|██████████| 5335/5335 [03:37<00:00, 24.57it/s]
